In [10]:
def load_dicom_volume_as_numpy_from_list(dcmlist):
    """Loads a list of PyDicom objects a Numpy array.
    Assumes that only one series is in the array

    Arguments:
        dcmlist {list of PyDicom objects} -- path to directory

    Returns:
        tuple of (3D volume, header of the 1st image)
    """

    # In the real world you would do a lot of validation here
    slices = [np.flip(dcm.pixel_array).T for dcm in sorted(dcmlist, key=lambda dcm: dcm.InstanceNumber)]

    # Make sure that you have correctly constructed the volume from your axial slices!
    hdr = dcmlist[0]

    # We return header so that we can inspect metadata properly.
    # Since for our purposes we are interested in "Series" header, we grab header of the
    # first file (assuming that any instance-specific values will be ighored - common approach)
    # We also zero-out Pixel Data since the users of this function are only interested in metadata
    hdr.PixelData = None
    return (np.stack(slices, 2), hdr)

In [11]:
def get_predicted_volumes(pred):
    """Gets volumes of two hippocampal structures from the predicted array

    Arguments:
        pred {Numpy array} -- array with labels. Assuming 0 is bg, 1 is anterior, 2 is posterior

    Returns:
        A dictionary with respective volumes
    """

    # Compute the volume of your hippocampal prediction
    volume_ant = np.sum(pred == 1)
    volume_post = np.sum(pred == 2)
    total_volume = volume_ant + volume_post

    return {"anterior": volume_ant, "posterior": volume_post, "total": total_volume}

In [12]:
def create_report(inference, header, orig_vol, pred_vol):
    """Generates an image with inference report

    Arguments:
        inference {Dictionary} -- dict containing anterior, posterior and full volume values
        header {PyDicom Dataset} -- DICOM header
        orig_vol {Numpy array} -- original volume
        pred_vol {Numpy array} -- predicted label

    Returns:
        PIL image
    """

    # The code below uses PIL image library to compose an RGB image that will go into the report
    # A standard way of storing measurement data in DICOM archives is creating such report and
    # sending them on as Secondary Capture IODs (http://dicom.nema.org/medical/dicom/current/output/chtml/part03/sect_A.8.html)
    # Essentially, our report is just a standard RGB image, with some metadata, packed into 
    # DICOM format. 

    pimg = Image.new("RGB", (1000, 1000))
    draw = ImageDraw.Draw(pimg)

    header_font = ImageFont.truetype("assets/Roboto-Regular.ttf", size=40)
    main_font = ImageFont.truetype("assets/Roboto-Regular.ttf", size=20)

    slice_nums = [orig_vol.shape[2]//3, orig_vol.shape[2]//2, orig_vol.shape[2]*3//4] # is there a better choice?
#     vol_slice = pred_vol[slice_nums[0],:slice_nums[1],:slice_nums[2]]

    # Create the report here and show information relevant to
    # clinicians.

    draw.text((10, 0), "HippoVolume.AI", (255, 255, 255), font=header_font)
    draw.multiline_text((10, 90), 
                        f"Patient ID: {header.PatientID}\n" \
                        f"Modality: {header.Modality}\n" \
                        f"Assistive AI Device: {header.SeriesDescription}\n" \
                        f"Inference Date: {header.StudyDate}\n\n" \
                        f"Hippocampus Volumes (cubic mm): \n" \
                        f"  Anterior Volume:   {inference['anterior']}\n" \
                        f"  Posterier Volume:  {inference['posterior']}\n" \
                        f"  Total Volume:      {inference['total']}\n\n" \
                        f"Volume render shown below. Hippocampus segmentation shown in green.",
                        (255, 255, 255), font=main_font)

    # Volume Rendering of Hippocampus
    vr_pred = np.zeros((pred_vol.shape[1], pred_vol.shape[2]))
    for z in range (pred_vol.shape[0]):
        vr_pred += pred_vol[z,:,:]
        
    vr_orig = np.zeros((orig_vol.shape[1], orig_vol.shape[2]))
    for z in range (orig_vol.shape[0]):
        vr_orig += orig_vol[z,:,:]
        
    # Create a PIL image from array:
    # Numpy array needs to flipped, transposed and normalized to a matrix of values in the range of [0..255]
    nd_img_pred = np.flip((vr_pred/np.max(vr_pred))*0xff).T.astype(np.uint8)
    nd_img_orig = np.flip((vr_orig/np.max(vr_orig))*0xff).T.astype(np.uint8)

    # This is how you create a PIL image from numpy array
    pil_i_pred = Image.fromarray(nd_img_pred, mode="L").convert("RGBA").resize((500,500))
    pil_i_orig = Image.fromarray(nd_img_orig, mode="L").convert("RGBA").resize((500,500))
    
    # Show prediction in different color
    # Split prediction into 3 channels
    r, g, b, a = pil_i_pred.split()
    # Remove Reds, Blues
    r = r.point(lambda i: i * 0)
    b = b.point(lambda i: i * 0)

    # Recombine back to RGB image
    result_pred = Image.merge('RGB', (r, g, b))
    
    # Paste the PIL image into our main report image object (pimg)
    pimg.paste(pil_i_orig, box=(10,400))
    pimg.paste(result_pred, box=(10,400), mask=Image.fromarray(nd_img_pred, mode="L").resize((500,500)))

    return pimg

In [13]:
def save_report_as_dcm(header, report, path):
    """Writes the supplied image as a DICOM Secondary Capture file

    Arguments:
        header {PyDicom Dataset} -- original DICOM file header
        report {PIL image} -- image representing the report
        path {Where to save the report}

    Returns:
        N/A
    """

    # Code below creates a DICOM Secondary Capture instance that will be correctly
    # interpreted by most imaging viewers including our OHIF
    # The code here is complete as it is unlikely that as a data scientist you will 
    # have to dive that deep into generating DICOMs. However, if you still want to understand
    # the subject, there are some suggestions below

    # Set up DICOM metadata fields. Most of them will be the same as original file header
    out = pydicom.Dataset(header)

    out.file_meta = pydicom.Dataset()
    out.file_meta.TransferSyntaxUID = pydicom.uid.ExplicitVRLittleEndian

    # Refer to this part of the standard to see what are the requirements for the valid
    # Secondary Capture IOD: http://dicom.nema.org/medical/dicom/2019e/output/html/part03.html#sect_A.8
    # The Modules table (A.8-1) contains a list of modules with a notice which ones are mandatory (M)
    # and which ones are conditional (C) and which ones are user-optional (U)
    # Note that we are building an RGB image which would have three 8-bit samples per pixel
    # Also note that writing code that generates valid DICOM has a very calming effect
    # on mind and body :)

    out.is_little_endian = True
    out.is_implicit_VR = False

    # We need to change class to Secondary Capture
    out.SOPClassUID = "1.2.840.10008.5.1.4.1.1.7"
    out.file_meta.MediaStorageSOPClassUID = out.SOPClassUID

    # Our report is a separate image series of one image
    out.SeriesInstanceUID = pydicom.uid.generate_uid()
    out.SOPInstanceUID = pydicom.uid.generate_uid()
    out.file_meta.MediaStorageSOPInstanceUID = out.SOPInstanceUID
    out.Modality = "OT" # Other
    out.SeriesDescription = "HippoVolume.AI"

    out.Rows = report.height
    out.Columns = report.width

    out.ImageType = r"DERIVED\PRIMARY\AXIAL" # We are deriving this image from patient data
    out.SamplesPerPixel = 3 # we are building an RGB image.
    out.PhotometricInterpretation = "RGB"
    out.PlanarConfiguration = 0 # means that bytes encode pixels as R1G1B1R2G2B2... as opposed to R1R2R3...G1G2G3...
    out.BitsAllocated = 8 # we are using 8 bits/pixel
    out.BitsStored = 8
    out.HighBit = 7
    out.PixelRepresentation = 0

    # Set time and date
    dt = datetime.date.today().strftime("%Y%m%d")
    tm = datetime.datetime.now().strftime("%H%M%S")
    out.StudyDate = dt
    out.StudyTime = tm
    out.SeriesDate = dt
    out.SeriesTime = tm

    out.ImagesInAcquisition = 1

    # We empty these since most viewers will then default to auto W/L
    out.WindowCenter = ""
    out.WindowWidth = ""

    # Data imprinted directly into image pixels is called "burned in annotation"
    out.BurnedInAnnotation = "YES"

    out.PixelData = report.tobytes()

    pydicom.filewriter.dcmwrite(path, out, write_like_original=False)

In [14]:


def get_series_for_inference(path):
    """Reads multiple series from one folder and picks the one
    to run inference on.

    Arguments:
        path {string} -- location of the DICOM files

    Returns:
        Numpy array representing the series
    """

    # Here we are assuming that path is a directory that contains a full study as a collection
    # of files
    # We are reading all files into a list of PyDicom objects so that we can filter them later

    dicoms = [pydicom.dcmread(os.path.join(path, f)) for f in os.listdir(path)]
    print('dicoms',dicoms)

    # Create a series_for_inference variable that will contain a list of only 
    # those PyDicom objects that represent files that belong to the series that you 
    # will run inference on.
    # It is important to note that radiological modalities most often operate in terms
    # of studies, and it will most likely be on you to establish criteria for figuring 
    # out which one of the multiple series sent by the scanner is the one you need to feed to 
    # your algorithm. In our case it's rather easy - we have reached an agreement with 
    # people who configured the HippoCrop tool and they label the output of their tool in a 
    # certain way.

    # Metadata contains the text "HippoCrop" in SeriesDescription field
    series_for_inference = []
    for d in dicoms:
        if (d[(0x0008,0x103e)].repval) == "'HippoCrop'":
            series_for_inference.append(d)
            
    # Check if there are more than one series (using set comprehension).
    if len({f.SeriesInstanceUID for f in series_for_inference}) != 1:
        print("Error: can not figure out what series to run inference on")
        return []
    
    return series_for_inference

In [15]:
def os_command(command):
    # Comment this if running under Windows
    #sp = subprocess.Popen(["/bin/bash", "-i", "-c", command])
    #sp.communicate()

    # Uncomment this if running under Windows
     os.system(command)

In [16]:
if __name__ == "__main__":
    # This code expects a single command line argument with link to the directory containing
    # routed studies
    if len(sys.argv) != 2:
        print("You should supply one command line argument pointing to the routing folder. Assuming ./studyimages dir.")
        sys.argv[1]="./studyimages/study1"

    # Find all subdirectories within the supplied directory. We assume that 
    # one subdirectory contains a full study
    subdirs = [os.path.join(sys.argv[1], d) for d in os.listdir(sys.argv[1]) if
                os.path.isdir(os.path.join(sys.argv[1], d))]
    
   
    # Get the latest directory
    study_dir = sorted(subdirs, key=lambda dir: os.stat(dir).st_mtime, reverse=True)[0]

    print(f"Looking for series to run inference on in directory {study_dir}...")
    print('study_dir',study_dir)
    number=get_series_for_inference(study_dir)
    print ('number',number)
    number
    volume, header = load_dicom_volume_as_numpy_from_list(get_series_for_inference(study_dir))
    # print('volume, header',volume, header)
    print(f"Found series of {volume.shape[2]} axial slices")
    print("HippoVolume.AI: Running inference...")
    # Use the UNetInferenceAgent class and model parameter file from the previous section
    inference_agent = UNetInferenceAgent(
        device="cpu",
        parameter_file_path=r"C:\Users\sethr\udacity\AIHippocampusSubmission\model.pth")

    # Run inference    
    pred_label = inference_agent.single_volume_inference_unpadded(volume)
    
    pred_volumes = get_predicted_volumes(pred_label)

    # Create and save the report
    print("Creating and pushing report...")
    report_save_path = r"./inference_report.dcm"
    report_img = create_report(pred_volumes, header, volume, pred_label)
    save_report_as_dcm(header, report_img, report_save_path)

    # Send report to our storage archive
    # Command line string that will issue a DICOM C-STORE request to send our report
    # to our Orthanc server (that runs on port 4242 of the local machine), using storescu tool
    
    os_command("storescu 127.0.0.1 4242 -v -aec HIPPOAI ./inference_report.dcm")

    
    
    # This line will remove the study dir if run as root user
    # Sleep to let our StoreSCP server process the report (remember - in our setup
    # the main archive is routing everything that is sent to it, including our freshly generated
    # report) - we want to give it time to save before cleaning it up
    time.sleep(2)
    shutil.rmtree(study_dir, onerror=lambda f, p, e: print(f"Error deleting: {e[1]}"))

    print(f"Inference successful on {header['SOPInstanceUID'].value}, out: {pred_label.shape}",
          f"volume ant: {pred_volumes['anterior']}, ",
          f"volume post: {pred_volumes['posterior']}, total volume: {pred_volumes['total']}")


You should supply one command line argument pointing to the routing folder. Assuming ./studyimages dir.
Looking for series to run inference on in directory ./studyimages/study1\13_HCropVolume...
study_dir ./studyimages/study1\13_HCropVolume
dicoms [Dataset.file_meta -------------------------------
(0002, 0000) File Meta Information Group Length  UL: 262
(0002, 0001) File Meta Information Version       OB: b'\x00\x01'
(0002, 0002) Media Storage SOP Class UID         UI: MR Image Storage
(0002, 0003) Media Storage SOP Instance UID      UI: 1.2.826.0.1.3680043.8.498.11676343796944751401139076219903216239
(0002, 0010) Transfer Syntax UID                 UI: Explicit VR Little Endian
(0002, 0012) Implementation Class UID            UI: 1.2.826.0.1.3680043.8.498.76647264891085702040065110813461770521
(0002, 0013) Implementation Version Name         SH: 'PYDICOM 1.4.0'
(0002, 0016) Source Application Entity Title     AE: 'DicomBrowser'
-------------------------------------------------
(0008, 

Creating and pushing report...
Inference successful on 1.2.826.0.1.3680043.8.498.11213494676556181167012876342158629840, out: (64, 64, 64) volume ant: 1959,  volume post: 1626, total volume: 3585
